## 1. From requirement to *user story*

Raw requirements often arrive as vague conversations: *“We need a report for finance.”*  A **user story** reframes that nebulous ask into a concise, testable sentence using the template: **As a `<role>`, I want `<feature>` so that `<benefit>`.**

Why bother?
- Forces you to **name the beneficiary** (user or system actor).
- Narrows scope to a single *behavior* rather than a bundle of hidden tasks.
- Provides a built‑in acceptance test: if the named role can’t gain the stated benefit, the story isn’t done.

Common gotcha → jumping straight to UI details (*“Add a green button”*) before clarifying the underlying goal.  Button color is design; the story is about *capability*.

```text
Conversation fragment:
    CFO: "Our auditors need proof that every invoice was sent to the client."

Bad story:
    "Add an export button to download all invoices as PDF."

Better story:
    As a *finance auditor*, I want to *receive a consolidated PDF bundle of all monthly invoices* so that *I can verify delivery compliance during audits*.
```

### Quick check

1. True / False A user story must specify **how** the feature will be implemented.

2. Which element is **missing** in the bad story above?
  a. Role   b. Benefit   c. Feature

<details><summary>Answer key</summary>

1. **False** — implementation details live elsewhere (tasks, design docs).
2. **b** — it lacks the explicit benefit/why.

</details>

## 2. Writing a *problem statement* & defining success criteria

A problem statement captures the gap between current state and desired outcome in one paragraph.  It should include **context**, **pain**, and **impact**.  After that, you list **success criteria**—measurable conditions that prove the gap is closed.

Benefits:
* Aligns team on *what “done” means* before any code.
* Gives QA and stakeholders an objective checklist.

Gotcha: statements that embed the presumed solution ("We need Kubernetes"), which blurs problem vs. implementation.

```text
Problem statement
-----------------
Right now customer support reps must open each ticket individually to see if the client has an active subscription plan.  This manual lookup takes ~30 seconds per ticket and creates frustration for callers who must wait on hold.

Success criteria
----------------
- The agent can see subscription status < 2 s after ticket open.
- Hold time attributable to plan lookups drops by 80 % within one month.
- Feature works for ≥ 95 % of tickets in the staging data set.
```

### Quick check

1. Which of these is a **success criterion**, not part of the problem statement?
  a. "Agents wait 30 s per lookup"  b. "Lookup time under 2 s"

2. True / False Including numeric targets in success criteria makes them less useful because they may be hard to hit.

<details><summary>Answer key</summary>

1. **b** — it states the desired measurable future.
2. **False** — numeric targets create clarity and testability.

</details>

## 3. Input → Processing → Output (IPO) frame

A classic way to reason about any program: break it into three buckets:
1. **Input** — data from users, files, APIs.
2. **Processing** — pure computations or transformations.
3. **Output** — what the system produces: files, DB rows, UI, network calls.

Why it matters:
* Keeps you from mixing side‑effects with logic, which simplifies testing.
* Helps spot missing pieces (e.g., we planned output but no input path).
* Encourages *pure* functions in the processing step—easier to cache & parallelize.

```python
# Mini IPO example: Celsius→Fahrenheit batch converter
def read_inputs(path):                # Input
    with open(path) as f:
        return [float(line) for line in f]

def c_to_f(c):                        # Processing (pure)
    return c * 9/5 + 32

def write_outputs(values, path):      # Output
    with open(path, 'w') as f:
        for v in values:
            f.write(f"{v:.1f}\n")

temps_c = read_inputs('celsius.txt')
temps_f = [c_to_f(t) for t in temps_c]
write_outputs(temps_f, 'fahrenheit.txt')
```

### Quick check

1. Which part of the IPO example is **easiest to unit‑test** in isolation?

2. True / False Combining reading, converting, and writing in one big function makes caching the conversion results easier.

<details><summary>Answer key</summary>

1. The *processing* function `c_to_f`—pure, deterministic, no I/O.
2. **False** — side‑effects tie the function to I/O, hindering reuse and caching.

</details>

## 4. Tracing with paper & pencil (desk‑checking)

Before running code, step through it manually on a small data set.  This old‑school habit uncovers off‑by‑one errors, wrong initialisations, and reveals whether you truly understand the algorithm.

Technique:
1. Draw columns for each variable.
2. Write initial values.
3. For every line of code, update the table.

Gotcha: skipping branch conditions—always mark whether the ‘if’ was taken or not.

```text
Code:
    total = 0
    for n in [1, 2, 3]:
        total += n

Trace table:
+-------+-------+
|  n    | total |
+-------+-------+
| start |   0   |
|   1   |   1   |
|   2   |   3   |
|   3   |   6   |
+-------+-------+
```

### Quick check

1. True / False Desk‑checking is obsolete because modern debuggers show variable state.

2. The trace table helps spot:
  a. Typo in print formatting  b. Off‑by‑one summation error

<details><summary>Answer key</summary>

1. **False** — quick pencil tracing is faster than firing up a debugger for tiny snippets.
2. **b** — step‑by‑step totals reveal indexing/count errors.

</details>

## 5. Identifying nouns → verbs → data vs. behaviour

In early design discussions, underline **nouns** (candidate data objects) and **verbs** (candidate behaviours).  Nouns often map to classes or DB tables; verbs to methods or service calls.

Example conversation: *“The **customer** uploads a **photo**, the system **scales** it and **stores** the thumbnail.”*
- Nouns → *customer*, *photo*, *thumbnail*
- Verbs → *upload*, *scale*, *store*

Gotcha: sometimes a verb becomes a noun (*upload* queue) or vice versa, so iterate.

```python
class Photo:
    def __init__(self, original_bytes):
        self.original = original_bytes
        self.thumbnail = None

    def scale(self):
        # behaviour attached to data
        self.thumbnail = self.original[:10]  # fake resize
```

### Quick check

1. Turning every noun into a class risks:
  a. anemic domain model  b. class explosion

2. True / False Methods that don’t use `self` should probably live outside the class.

<details><summary>Answer key</summary>

1. **b** — too many tiny classes increase complexity.
2. **True** — they’re utility functions, better as module‑level helpers.

</details>